In [1]:
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords, ALL_TIMES,interpline, CoordPair, xy_to_ll, ll_to_xy)
import pandas as pd
import numpy as np
import os
import shutil
import re
import netCDF4 as nc
import xarray as xr

In [2]:
#mask out seas-1620

ds_lu_1620 = xr.open_dataset('wrfout_d04_2011-06-23_11:00:00')
lu_1620 = ds_lu_1620['LU_INDEX']

lu_1620 = lu_1620[0,:,:]

del lu_1620.coords['XTIME']

lu_1620_df = lu_1620.to_dataframe().reset_index()

In [3]:
#mask out seas-2040s

ds_lu_2040s = xr.open_dataset('wrfout_d04_2040-06-18_23:00:00')
lu_2040s = ds_lu_2040s['LU_INDEX']

lu_2040s = lu_2040s[0,:,:]

del lu_2040s.coords['XTIME']

lu_2040s_df = lu_2040s.to_dataframe().reset_index()

# 2010s

In [4]:
ds1 = xr.open_dataset('ssp245_t2_1120_era5_corrected_dailymax.nc')
t2_1120_ssp245_dailymax = ds1['ssp245_t2_1120_era5_corrected_dailymax']

t2_1120_ssp245_dailymax_df = t2_1120_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_1120_dailymax_thres = (t2_1120_ssp245_dailymax_df[t2_1120_ssp245_dailymax_df['ssp245_t2_1120_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [5]:
#pick out days with maximum temperature over 33℃
HDF_1120 = t2_1120_dailymax_thres.groupby(['XLAT','XLONG','west_east','south_north'],as_index=False)['ssp245_t2_1120_era5_corrected_dailymax'].count()
HDF_1120 = HDF_1120.rename(columns={'ssp245_t2_1120_era5_corrected_dailymax':'hot_days_num'})

HDF_1120['hot_days_num'] = HDF_1120['hot_days_num']/10

In [6]:
HDF_1120['hot_days_num'].max()

96.3

In [7]:
#mask out the sea

HDF_1120 = pd.merge(HDF_1120,lu_1620_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HDF_1120['hot_days_num'] = HDF_1120['hot_days_num'].fillna(0)

HDF_1120.loc[HDF_1120.LU_INDEX == 16, 'hot_days_num'] = 9999

HDF_1120 = HDF_1120.rename(columns={"hot_days_num": "hdf"})

In [8]:
HDF_1120

,XLAT,XLONG,west_east,south_north,hdf,LU_INDEX
0,21.751648,112.761719,0,0,9999.0,16.0
1,21.751740,112.771606,1,0,9999.0,16.0
2,21.751839,112.781464,2,0,1.8,2.0
3,21.751923,112.791321,3,0,1.1,3.0
4,21.752014,112.801208,4,0,0.9,8.0
...,...,...,...,...,...,...
34501,23.176796,114.854218,211,155,0.0,15.0
34502,23.186295,114.804321,206,156,0.0,15.0
34503,23.186234,114.814331,207,156,0.0,7.0
34504,23.186165,114.824310,208,156,0.0,7.0


In [9]:
HDF_1120.to_csv('HDF_1120.csv',index=False,columns=['hdf','XLAT','XLONG'])

HDF_1120.to_csv('HDF_1120_withgrids.csv',index=False,columns=['XLAT','XLONG','hdf','south_north','west_east'])

# 2040s

In [10]:
ds1 = xr.open_dataset('ssp245_t2_4049_era5_corrected_dailymax.nc')
t2_4049_ssp245_dailymax = ds1['ssp245_t2_4049_era5_corrected_dailymax']

t2_4049_ssp245_dailymax_df = t2_4049_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_4049_dailymax_thres = (t2_4049_ssp245_dailymax_df[t2_4049_ssp245_dailymax_df['ssp245_t2_4049_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [11]:
#pick out days with maximum temperature over 33℃
HDF_4049 = t2_4049_dailymax_thres.groupby(['XLAT','XLONG','west_east','south_north'],as_index=False)['ssp245_t2_4049_era5_corrected_dailymax'].count()
HDF_4049 = HDF_4049.rename(columns={'ssp245_t2_4049_era5_corrected_dailymax':'hot_days_num'})

HDF_4049['hot_days_num'] = HDF_4049['hot_days_num']/10

In [12]:
#mask out the sea

HDF_4049 = pd.merge(HDF_4049,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HDF_4049['hot_days_num'] = HDF_4049['hot_days_num'].fillna(0)

HDF_4049.loc[HDF_4049.LU_INDEX == 16, 'hot_days_num'] = 9999

HDF_4049 = HDF_4049.rename(columns={"hot_days_num": "hdf"})

In [13]:
HDF_4049.to_csv('HDF_4049.csv',index=False,columns=['hdf','XLAT','XLONG'])

HDF_4049.to_csv('HDF_4049_withgrids.csv',index=False,columns=['XLAT','XLONG','hdf','south_north','west_east'])

# 2090s

In [14]:
ds1 = xr.open_dataset('ssp245_t2_9099_era5_corrected_dailymax.nc')
t2_9099_ssp245_dailymax = ds1['ssp245_t2_9099_era5_corrected_dailymax']

t2_9099_ssp245_dailymax_df = t2_9099_ssp245_dailymax.to_dataframe().reset_index()

#Use 33℃ as a threshold

t2_9099_dailymax_thres = (t2_9099_ssp245_dailymax_df[t2_9099_ssp245_dailymax_df['ssp245_t2_9099_era5_corrected_dailymax']>=33]).reset_index(drop=True)

In [15]:
#pick out days with maximum temperature over 33℃
HDF_9099 = t2_9099_dailymax_thres.groupby(['XLAT','XLONG','west_east','south_north'],as_index=False)['ssp245_t2_9099_era5_corrected_dailymax'].count()
HDF_9099 = HDF_9099.rename(columns={'ssp245_t2_9099_era5_corrected_dailymax':'hot_days_num'})

HDF_9099['hot_days_num'] = HDF_9099['hot_days_num']/10

In [16]:
#mask out the sea

HDF_9099 = pd.merge(HDF_9099,lu_2040s_df,on=['XLAT','XLONG','south_north','west_east'],how="outer")

HDF_9099['hot_days_num'] = HDF_9099['hot_days_num'].fillna(0)

HDF_9099.loc[HDF_9099.LU_INDEX == 16, 'hot_days_num'] = 9999

HDF_9099 = HDF_9099.rename(columns={"hot_days_num": "hdf"})

In [17]:
HDF_9099.to_csv('HDF_9099.csv',index=False,columns=['hdf','XLAT','XLONG'])

HDF_9099.to_csv('HDF_9099_withgrids.csv',index=False,columns=['XLAT','XLONG','hdf','south_north','west_east'])

# diff

In [18]:
HDF_1120 = pd.read_csv('HDF_1120_withgrids.csv')

HDF_1120 = HDF_1120.replace(9999,np.nan)

HDF_1120 = HDF_1120.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [19]:
HDF_4049 = pd.read_csv('HDF_4049_withgrids.csv')

HDF_4049 = HDF_4049.replace(9999,np.nan)

HDF_4049 = HDF_4049.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [20]:
HDF_9099 = pd.read_csv('HDF_9099_withgrids.csv')

HDF_9099 = HDF_9099.replace(9999,np.nan)

HDF_9099 = HDF_9099.sort_values(['west_east','south_north'],ascending=True).reset_index(drop=True)

In [21]:
#test

print((HDF_1120['west_east']-HDF_4049['west_east']).min())
print((HDF_1120['west_east']-HDF_4049['west_east']).max())
print((HDF_1120['south_north']-HDF_4049['south_north']).max())
print((HDF_1120['south_north']-HDF_4049['south_north']).min())
print((HDF_4049['west_east']-HDF_9099['west_east']).max())
print((HDF_4049['west_east']-HDF_9099['west_east']).min())
print((HDF_4049['south_north']-HDF_9099['south_north']).max())
print((HDF_4049['south_north']-HDF_9099['south_north']).min())

0
0
0
0
0
0
0
0


In [22]:
HDF_diff_2040s = HDF_1120.copy()
HDF_diff_2090s = HDF_1120.copy()

In [23]:
HDF_diff_2040s['diff'] = HDF_4049['hdf']-HDF_1120['hdf']

HDF_diff_2090s['diff'] = HDF_9099['hdf']-HDF_1120['hdf']

In [24]:
HDF_diff_2040s['diff'] = HDF_diff_2040s['diff'].fillna(9999)

HDF_diff_2090s['diff'] = HDF_diff_2090s['diff'].fillna(9999)

In [25]:
HDF_diff_2040s.to_csv('HDF_diff_2040s.csv')
HDF_diff_2090s.to_csv('HDF_diff_2090s.csv')